In [1]:
import requests
import json
from datetime import datetime
from sklearn.cluster import KMeans
import scipy.optimize as optimize
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 100000)
pd.set_option("display.precision", 12)
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_data(endpointApi, date=None, full_response=False, columns=list()):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    payload = {'date': date, 'fullResponse': full_response}
    res = requests.get(URL + endpointApi,
                      params=payload,
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

In [3]:
def put_data(endpointApi, content):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    res = requests.put(url=URL + endpointApi,
                      data=json.dumps(content),
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

In [4]:
def post_data(endpointApi, content):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    res = requests.post(url=URL + endpointApi,
                      data=content,
                      auth=AUTH,
                      headers = {"content-type": "application/json"},
                      verify=False)
    return res.content.decode('utf-8')

Conversion de chaque valeur en le type approprié :

In [5]:
def convert_currency(value, df_currency_rate):
    value, currency_name = value.split(' ')
    rate = currencyRateToEUR(currency_name, df_currency_rate)
    
    new_value = float(value.replace(',', '.')) * rate
    return new_value

def convert_datetime(value):
    return datetime.strptime(value, '%Y-%m-%d').date()

def convert_float(value):
    return float(value.replace(',', '.'))

def convert_int(value):
    return int(value)

def convert_boolean(value):
    return json.loads(value)

def df_convert_type(df, df_currency_rate):
        for col in df.columns:
            convert_values = []
            for elt in df[col]:
                if elt is np.nan:
                    convert_values.append(np.nan)
                    continue

                elt_type = elt['type']
                elt_value = elt['value']
                if elt_type == 'currency_value':
                    elt_value = convert_currency(elt_value, df_currency_rate)
                elif elt_type == 'date':
                    elt_value = convert_datetime(elt_value)
                elif elt_type in ['double', 'percent']:
                    elt_value = convert_float(elt_value)
                elif elt_type in ['asset', 'int32']:
                    elt_value = convert_int(elt_value)
                elif elt_type == 'boolean':
                    elt_value = convert_boolean(elt_value)
                elif elt_type not in ['asset_type', 'string', 'asset_currency', 'date-time',
                                     'asset_sub_type', 'asset_status', 'asset_quote_type',
                                     'liquidity_algorithm', 'portfolio_lock_mode', 'portfolio_type']:
                    print(elt)
                convert_values.append(elt_value)
            df[col] = convert_values
        return df

Création de la table des taux en EUR : 

In [6]:
def getCurrencyRate():
    d = []
    currency_json = json.loads(get_data('currency'))
    for currency in currency_json:
        currency_id = currency.get('id')
        rate = get_data('currency/rate/{}/to/EUR'.format(currency_id))
        if len(rate) == 0:
            continue
        rate_value = json.loads(rate)['rate']['value']
        d.append([currency_id, convert_float(rate_value)])
    return pd.DataFrame(d, columns=['currency src', 'rate'])
    
df_currency_rate = getCurrencyRate()
df_currency_rate

,currency src,rate
0,USD,0.851208716377
1,EUR,1.000000000000
2,JPY,0.008066467694
3,BEF,0.024789352477
4,CYP,1.708601441376
5,EEK,0.063911648537
6,MTL,2.329373398556
7,SKK,0.033193918874
8,USN,0.851208716377
9,USS,0.851208716377


Taux de conversion entre une monnaie en input et l'euro :

In [7]:
def currencyRateToEUR(cur_src, df_currency_rate):
    return df_currency_rate[df_currency_rate['currency src'] == cur_src]['rate'].values[0]

La base de données complète de tous les actifs :

In [8]:
data = get_data('asset/')
df = pd.read_json(data)
df_assets = df_convert_type(df, df_currency_rate)
df_assets

Output hidden; open in https://colab.research.google.com to view.

Récupération de la liste des actifs en base :

In [9]:
def get_df_asset(asset_date, df_currency_rate, asset_id='ASSET_DATABASE_ID', asset_label='LABEL',
                 asset_type='TYPE', asset_last_close_value='LAST_CLOSE_VALUE_IN_CURR', asset_min_buy_amount='MIN_BUY_AMOUNT'):
    data = get_data('asset?columns={}&columns={}&columns={}&columns={}&columns={}&columns={}&date={}&columns=CREATION_DATE'
                   .format(asset_id, asset_label, asset_type, asset_last_close_value, asset_min_buy_amount, 'asset_fund_info_decimalisation', asset_date))
    df_asset = pd.read_json(data)
    df_asset = df_convert_type(df_asset, df_currency_rate)
    df_asset = df_asset[(df_asset['LAST_CLOSE_VALUE_IN_CURR'].notna()) | (df_asset['TYPE'] == 'PORTFOLIO')].reset_index(drop=True)
    return df_asset

df_ETF = get_df_asset('2016-06-01', df_currency_rate)
df_ETF

,LABEL,LAST_CLOSE_VALUE_IN_CURR,CREATION_DATE,ASSET_DATABASE_ID,TYPE,asset_fund_info_decimalisation,MIN_BUY_AMOUNT
0,1&1 DRILLISCH AG,3.685100000000e+01,2019-10-02,1845,STOCK,NaN,NaN
1,1000MERCIS,4.090000000000e+01,2019-10-02,1846,STOCK,NaN,NaN
2,1818-VEGA EUR CO,3.937000000000e+01,2019-10-02,2122,FUND,NaN,NaN
3,29 HAUSSMANN CROISSANCE EUROPE D,1.670650000000e+03,2019-10-02,2123,FUND,3.0,0.000000000000
4,AALBERTS INDS,3.170000000000e+01,2019-10-02,1847,STOCK,NaN,NaN
5,AAREAL BANK AG,3.281000000000e+01,2019-10-02,1848,STOCK,NaN,NaN
6,ABC ARBITRAGE,6.730000000000e+00,2019-10-02,1849,STOCK,NaN,NaN
7,ABERDEEN CHINA EQUITY A2,1.786653047327e+01,2019-10-02,2154,FUND,NaN,1276.813074565500
8,ABERDEEN GLOBAL - EMERGING MARKETS BOND - A AC...,3.371237657473e+01,2019-10-02,2156,FUND,NaN,1276.813074565500
9,ABER-GL JP E-A2=,1.176730000000e+01,2019-10-02,2127,FUND,NaN,1500.000000000000


Récupération des valeurs de cotations:

In [10]:
def get_cotation_values(asset_id, start_date, end_date, df_currency_rate):
    data = get_data('asset/{}/quote?start_date={}&end_date={}'
                    .format(asset_id, start_date, end_date))
    df_cotation = pd.read_json(data)
    df_cotation = df_convert_type(df_cotation, df_currency_rate)
    return df_cotation

Récupération de la liste des ratios :

In [11]:
def get_ratio(df_currency_rate):
    data = get_data('ratio')
    df_ratio = pd.read_json(data)
    return df_ratio

df_ratio = get_ratio(df_currency_rate)
df_ratio

,id,type,name,is_benchmark_needed,is_percent
0,11,Ratio,Correlation,True,False
1,7,Ratio,Exposition action,False,True
2,15,Ratio,Exposition action,False,True
3,8,Ratio,RDT,False,True
4,13,Ratio,Rendement,False,True
5,9,Ratio,Rendement_annualise,False,True
6,12,Ratio,Sharpe,False,False
7,14,Ratio,VaR historique,False,True
8,10,Ratio,Volatilite,False,True


Calcul des ratios :

In [12]:
def post_ratio(ratios, assets_id, start_date, end_date, bench='null', frequency=None):
    payload = "{{'ratio'={},'asset'={},'start_date'={},'end_date'={}, 'benchmark'={}, 'frequency':null}}".format(ratios, assets_id, start_date, end_date, bench)
    reponse = post_data('ratio/invoke', payload)
    return reponse

In [13]:
def get_df_ratios(assets_id, begin_date, end_date):
    json_ratios = json.loads(post_ratio([10, 12, 13], assets_id, begin_date, end_date))
    ratios = []
    for asset_id in assets_id:
        str_asset_id = str(asset_id)
        volatilite = convert_float(json_ratios[str_asset_id]['10']['value'])
        sharpe = convert_float(json_ratios[str_asset_id]['12']['value'])
        rendement = convert_float(json_ratios[str_asset_id]['13']['value'])
        ratios.append([asset_id, volatilite, rendement, sharpe])
    return pd.DataFrame(ratios, columns=['ASSET_DATABASE_ID', 'volatilite', 'rendement', 'sharpe'])

Recupération du portfolio ID :

In [14]:
def get_portfolio_id(df_assets, label):
    portfolio_id = df_assets.loc[(df_assets['TYPE'] == 'PORTFOLIO') & (df_assets['LABEL'] == label)]['REST_OBJECT_ID'].values[0]
    return int(portfolio_id)

Récupération de la composition d’un portefeuille : 

In [15]:
def get_portfolio(portfolio_id):
    data = get_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id))
    df_portfolio = pd.read_json(data)
    return df_portfolio

Récupération des actifs d'un portefeuille :

In [16]:
def get_assets_portfolio(df_portfolio, date):
    if date not in df_portfolio['values']:
        return np.NaN
    return df_portfolio['values'][date]

Mise à jour de la composition d’un portefeuille :


In [17]:
def portfolio_json_assets(assets):
    json = ''
    for asset in assets:
        json += '{{"asset":{{"asset": {}, "quantity": {}}}}},'.format(asset["asset"]["asset"], asset["asset"]["quantity"])
    return json[:-1]
    
def seralize_portfolio_content(df_portfoliot, assets=None):
    label = df_portfoliot['label'][0]
    currency = df_portfoliot['currency'][0]
    type_ = df_portfoliot['type'][0]
    date = '2016-06-01'

    if assets is None:
      res = '{{"label": "{}", "currency": {{"code": "{}"}}, "type": "{}", "values": {{}} }}'.format(label, currency, type_, date)

    else:
      assets = portfolio_json_assets(assets)
      res = '{{"label": "{}", "currency": {{"code": "{}"}}, "type": "{}", "values": {{"{}": [{}]}}}}'.format(label, currency, type_, date, assets)
    return json.loads(res)

def deseralize_portfolio_content(json_portfolio):
    json_portfolio = pd.json_normalize(json_portfolio)
    
    cols = [col for col in json_portfolio.columns if col.split('.')[0] == 'values']
    json_portfolio = json_portfolio[cols]
    json_portfolio.columns = json_portfolio.columns.map(lambda x: x.split(".")[0])

    json_assets = json_portfolio['values'][0]
    assets = []
    for asset in json_assets:
        assets.append(asset)
    return assets

Fonction pour écrire la composition d'actifs dans le portefeuille :

In [18]:
def put_portfolio(portfolio_id, df_portfolio, assets):
    content = seralize_portfolio_content(df_portfolio, assets)
    put_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id), content)

Réinitialisation des actifs à l'intérieur du portefeuille :

In [19]:
def reset_assets_portfolio(portfolio_id, df_portfolio):
    content = seralize_portfolio_content(df_portfolio)
    content['values'] = {}
    put_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id), content)

**SELECTION DES ACTIFS POUR LE PORTFOLIO :**

Création de la matrice de corrélation entre les actifs :

In [20]:
def get_correlation_matrix(assets_id, begin_date, end_date):
  df_correlation = pd.DataFrame({}, columns=assets_id)
  for asset_id in assets_id:
    correlation_ratio = json.loads(post_ratio([11], assets_id, begin_date, end_date, asset_id))
    df_correlation[asset_id] = [convert_float(correlation_ratio[str(asset)]['11']['value']) for asset in assets_id]
  return df_correlation.set_index([pd.Index(assets_id)])

#remove portfolios in assets list
assets = df_ETF
assets = assets[assets['TYPE'] != 'PORTFOLIO']
assets_id = [int(asset_id) for asset_id in assets['ASSET_DATABASE_ID'].values]

begin_date = '2016-06-01'
end_date = '2020-09-30'

correlation_matrix = get_correlation_matrix(assets_id, begin_date, end_date)

In [21]:
correlation_matrix

Output hidden; open in https://colab.research.google.com to view.

In [32]:
def select_less_correlate_assets(assets, assets_id, begin_date, end_date, nb_clusters):
  #correlation_matrix = get_correlation_matrix(assets_id, begin_date, end_date) #FIXME

  kmeans = KMeans(n_clusters=nb_clusters, random_state=5) #FIXME
  Y_pred = kmeans.fit_predict(correlation_matrix)
  
  df_kmeans_group_prediction = pd.DataFrame(Y_pred, columns=['group'])
  assets = assets.join(df_kmeans_group_prediction)

  for i in range(nb_clusters):
    best_sharpe = assets[assets['group'] == i].sort_values(by=['sharpe'], ascending=False)['sharpe'].values[0]
  assets_with_best_sharpe = [assets[assets['group'] == i].sort_values(by=['sharpe'], ascending=False)['ASSET_DATABASE_ID'].values[0] for i in range(nb_clusters)] #selectionne pour chaque groupe l'actif avec le meilleur sharpe

  assets = assets[assets['ASSET_DATABASE_ID'].isin(assets_with_best_sharpe)].reset_index(drop=True)
  return assets

Evaluation des ratios (rendement, volatilité et sharpe) du portfolio :

In [23]:
def portfolio_stats(assets, weights):
    returns = assets['rendement'].values
    volatilities = assets['volatilite'].values
    sharpes = assets['sharpe'].values

    total_returns = np.dot(returns, weights)
    total_volatilities = np.dot(volatilities, weights)

    total_sharpes = total_returns / total_volatilities
    
    return {'return': total_returns, 'volatility': total_volatilities, 'sharpe': total_sharpes}

Minimization Functions

In [24]:
def minimize_neg_sharpe(weights, assets):#on veut maximiser sharpe
    return -portfolio_stats(assets, weights)['sharpe'] 

La fonction pour trouver les poids optimaux et les fonctions de contraintes :

In [25]:
def nav_max_constraint(weights):
     assets = assets_global
     nb_weights = len(weights)

     max_nav = None
     for i in range(nb_weights):
       nav = weights[i] * assets['LAST_CLOSE_VALUE_IN_CURR'].values[i]
       if max_nav is None:
         max_nav = nav
       else:
         max_nav = max(max_nav, nav)
     return max_nav - np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.098  #FIXME #on réduit à en-dessous de 0.10 pour être large (res va converger vers 0 mais il ne sera jamais égale à 0)

def nav_min_constraint(weights):
     assets = assets_global
     nb_weights = len(weights)

     min_nav = None
     for i in range(nb_weights):
       nav = weights[i] * assets['LAST_CLOSE_VALUE_IN_CURR'].values[i]
       if min_nav is None:
         min_nav = nav
       else:
         min_nav = min(min_nav, nav)
     return min_nav - np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.012  #FIXME #on augmente à au-dessus de 0.01 pour être large (res va converger vers 0 mais il ne sera jamais égale à 0)

def stock_constraint(weights):
     assets = assets_global

     stock_assets_id = [asset_id for asset_id in assets[assets['TYPE'] == 'STOCK'].index]
     stock_weights = weights[stock_assets_id]
     stock_nav = assets['LAST_CLOSE_VALUE_IN_CURR'].values[stock_assets_id]

     return np.dot(stock_weights, stock_nav) - np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * stock_percent_global #on augmente au-dessus de 0.5 pour être large (res va converger vers 0 mais il ne sera jamais égale à 0)

def optimize_sharpe(assets, stock_percent):
    nb_assets = assets.shape[0]
    
    global assets_global
    assets_global = assets

    global stock_percent_global
    stock_percent_global = stock_percent
    
    constraints = ({'type': 'eq', 'fun': nav_min_constraint},
                   {'type': 'eq', 'fun': nav_max_constraint},
                   {'type': 'eq', 'fun': stock_constraint})
    bounds = tuple((1, None) for x in range(nb_assets))
    initializer = nb_assets * [1/ nb_assets]
    
    best_sharpe = optimize.minimize(minimize_neg_sharpe, initializer, args = (assets), method='SLSQP', bounds=bounds, constraints=constraints)
    return best_sharpe.x

Fonctions pour vérifier si notre configuration d'actifs respecte bien les contraintes du sujet :

In [26]:
def check_nav_max_constraint(assets):
    weights = assets['weights'].values
    nb_weights = len(weights)

    max_nav = None
    for i in range(nb_weights):
      nav = weights[i] * assets['LAST_CLOSE_VALUE_IN_CURR'].values[i]
      if max_nav is None:
        max_nav = nav
      else:
        max_nav = max(max_nav, nav)
    return max_nav <= np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.10
     
def check_nav_min_constraint(assets):
    weights = assets['weights'].values
    nb_weights = len(weights)

    min_nav = None
    for i in range(nb_weights):
      nav = weights[i] * assets['LAST_CLOSE_VALUE_IN_CURR'].values[i]
      if min_nav is None:
        min_nav = nav
      else:
        min_nav = min(min_nav, nav)
    return min_nav >= np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.01

def check_nav_constraint(assets):
     check_max = check_nav_max_constraint(assets)
     check_min = check_nav_min_constraint(assets)
     return check_max and check_min

def check_stock_constraint(assets):
     weights = assets['weights'].values
     
     stock_assets_index = [asset_index for asset_index in assets[assets['TYPE'] == 'STOCK'].index]
     stock_weights = weights[stock_assets_index]
     stock_nav = assets['LAST_CLOSE_VALUE_IN_CURR'].values[stock_assets_index]
     return np.dot(stock_weights, stock_nav) >= np.dot(weights, assets['LAST_CLOSE_VALUE_IN_CURR'].values) * 0.5

def check_quantity_constraint(assets):
    for _, row in assets.iterrows():
      if ((row['MIN_BUY_AMOUNT'] is not np.NaN) and (row['weights'] < row['MIN_BUY_AMOUNT'])):
          return False
    return True

def check_decimalisation_constraint(assets):
    return assets[assets['asset_fund_info_decimalisation'].notna()].empty


Fonction qui sélectionne la meilleure configuration d'actifs :

In [27]:
import scipy as sp
import scipy.optimize as scopt
import scipy.stats as spstats

def select_best_assets(begin_date, end_date, df_currency_rate, nb_assets, stock_percent):
    assets = get_df_asset(begin_date, df_currency_rate)
    #retire des actifs les deux portfolios
    assets = assets[assets['TYPE'] != 'PORTFOLIO']
    assets_id = [int(asset_id) for asset_id in assets['ASSET_DATABASE_ID'].values]
    
    #tableau de tous les ratios 12 Sharpe 13 Rendement 10 Volatile pour chaque actifs
    df_ratios = get_df_ratios(assets_id, begin_date, end_date)
    assets = assets.merge(df_ratios, on='ASSET_DATABASE_ID')

    #sélectionne les 15 meilleurs actifs après avoir regroupé les actifs en fonction de leurs corrélations grâce à un KMeans :
    best_assets = select_less_correlate_assets(assets, assets_id, begin_date, end_date, nb_assets)
    best_assets = best_assets.sort_values(by=['sharpe'], ascending=False)[:15].reset_index(drop=True)

    weights = optimize_sharpe(best_assets, stock_percent)
    best_assets['weights'] = (weights * 100).astype(int)

    constraints_respected = check_nav_constraint(best_assets) and check_stock_constraint(best_assets) and check_quantity_constraint(best_assets) and check_decimalisation_constraint(best_assets)
    return best_assets, constraints_respected

Poste la configuration d'actifs dans le portefeuille :

In [28]:
def post_assets_selected_in_portfolio(assets, begin_date, df_currency_rate):
    assets_id_quantity = [{"asset": {"asset": asset['ASSET_DATABASE_ID'], "quantity": asset['weights']}} for _, asset in assets.iterrows()]
    df_assets = df_convert_type(pd.read_json(get_data('asset/')), df_currency_rate)
    
    portfolio_id = get_portfolio_id(df_assets, 'EPITA_PTF_10')
    df_portfolio = get_portfolio(portfolio_id)
    
    reset_assets_portfolio(portfolio_id, df_portfolio)#on réinitialise le contenu du portefeuille (plus safe)
    
    df_portfolio = get_portfolio(portfolio_id)
    put_portfolio(portfolio_id, df_portfolio, assets_id_quantity)
    put_portfolio(portfolio_id, df_portfolio, assets_id_quantity)#on doit poster une seconde fois le portefeuille pour bien récupérer le ratio de sharpe (bug de l'API)

Comparaison du ratio de sharpe entre notre portefeuille et le portefeuille de référence : 

In [29]:
def compare_portfolios(our_portfolio_id, ref_portfolio_id, begin_date, end_date):    
    ratios = json.loads(post_ratio([12], [our_portfolio_id, ref_portfolio_id], begin_date, end_date))
    return ratios[str(our_portfolio_id)]['12']['value']

Création du portefeuille en maximisant le ratio de sharpe :

In [43]:
def create_our_best_portfolio(begin_date, end_date, nb_assets_groups_limit=0, stock_percent_limit=0):
  #df_currency_rate = getCurrencyRate()#FIXME
  df_assets = df_convert_type(pd.read_json(get_data('asset/')), df_currency_rate)
  our_portfolio_id = get_portfolio_id(df_assets, 'EPITA_PTF_10')
  ref_portfolio_id = get_portfolio_id(df_assets, 'REF')

  best_sharp_ratio = {'nb_groups': 25, 'stock_percent': 0.5129, 'sharp': None} #meilleur ratio de sharpe trouvé avec create_our_best_portfolio(begin_date, end_date, 26, 300)

  for nb_assets_groups in range(nb_assets_groups_limit):
      print('number assets groups = {}'.format(15 + nb_assets_groups))
      assets_selected, constraints_respected = select_best_assets(begin_date, end_date, df_currency_rate, 15 + nb_assets_groups, 0.55)
      
      #si les contraintes de composition de portfolio sont respectées alors on poste le portfolio
      if constraints_respected:
        post_assets_selected_in_portfolio(assets_selected, begin_date, df_currency_rate)
        our_sharp_ratio = compare_portfolios(our_portfolio_id, ref_portfolio_id, begin_date, end_date)

        if best_sharp_ratio['sharp'] is None or (our_sharp_ratio > best_sharp_ratio['sharp']):
            best_sharp_ratio = {'nb_groups': 15 + nb_assets_groups, 'stock_percent': 0.55, 'sharp': our_sharp_ratio}
            print('new best sharp ratio found : {}'.format(best_sharp_ratio))

  nb_assets_groups = best_sharp_ratio['nb_groups']
  print('Best number assets groups found = {}'.format(nb_assets_groups))

  for stock_percent in range(stock_percent_limit):
    print('stock percent = {}'.format(0.0001 * (5000 + stock_percent))) #FIXME
    assets_selected, constraints_respected = select_best_assets(begin_date, end_date, df_currency_rate, nb_assets_groups, 0.0001 * (5000 + stock_percent)) #FIXME
    
    #si les contraintes de composition de portfolio sont respectées alors on poste le portfolio
    if constraints_respected:
      post_assets_selected_in_portfolio(assets_selected, begin_date, df_currency_rate)
      our_sharp_ratio = compare_portfolios(our_portfolio_id, ref_portfolio_id, begin_date, end_date)

      if best_sharp_ratio['sharp'] is None or (our_sharp_ratio > best_sharp_ratio['sharp']):
          best_sharp_ratio = {'nb_groups': nb_assets_groups, 'stock_percent': 0.0001 * (5000 + stock_percent), 'sharp': our_sharp_ratio} #FIXME
          print('new best sharp ratio found : {}'.format(best_sharp_ratio))

  #poste la meilleure configuration trouvé:
  print('Best portfolio configuration found: {}'.format(best_sharp_ratio))
  assets_selected, constraints_respected = select_best_assets(begin_date, end_date, df_currency_rate, best_sharp_ratio['nb_groups'], best_sharp_ratio['stock_percent'])
  print('Constraints respected = {}'.format(constraints_respected))
  post_assets_selected_in_portfolio(assets_selected, begin_date, df_currency_rate)

  #comparaison du ratio de sharpe entre notre portefolio et le portefolio de référence
  our_sharp_ratio = compare_portfolios(our_portfolio_id, ref_portfolio_id, begin_date, end_date)
  print('Sharpe ratio in our portfolio : {}'.format(our_sharp_ratio))
  ratios = json.loads(post_ratio([12], [ref_portfolio_id], begin_date, end_date))
  print('Sharpe ratio in ref portfolio : {}'.format(ratios[str(ref_portfolio_id)]['12']['value']))

Fonction Main :

In [44]:
def main():
  begin_date = '2016-06-01'
  end_date = '2020-09-30'
  create_our_best_portfolio(begin_date, end_date, 26, 500)

main()

number assets groups = 15
new best sharp ratio found : {'nb_groups': 15, 'stock_percent': 0.55, 'sharp': '2,70608472514'}
number assets groups = 16
new best sharp ratio found : {'nb_groups': 16, 'stock_percent': 0.55, 'sharp': '2,712803472443'}
number assets groups = 17
number assets groups = 18
number assets groups = 19
number assets groups = 20
new best sharp ratio found : {'nb_groups': 20, 'stock_percent': 0.55, 'sharp': '2,906699636026'}
number assets groups = 21
number assets groups = 22
number assets groups = 23
new best sharp ratio found : {'nb_groups': 23, 'stock_percent': 0.55, 'sharp': '2,966508729494'}
number assets groups = 24
number assets groups = 25
new best sharp ratio found : {'nb_groups': 25, 'stock_percent': 0.55, 'sharp': '3,08771197545'}
number assets groups = 26
number assets groups = 27
number assets groups = 28
number assets groups = 29
number assets groups = 30
number assets groups = 31
number assets groups = 32
number assets groups = 33
number assets groups = 

ValueError: ignored

Vérification des contraintes du portefeuille :

In [36]:
  
df_assets = df_convert_type(pd.read_json(get_data('asset/')), df_currency_rate)
our_portfolio_id = get_portfolio_id(df_assets, 'EPITA_PTF_10')

df_assets = get_df_asset('2016-06-01', df_currency_rate)
df_portfolio = get_portfolio(our_portfolio_id)
df_portfolio

,label,currency,type,values
code,EPITA_PTF_10,EUR,front,NaN
2016-06-01,EPITA_PTF_10,NaN,front,"[{'asset': {'asset': 1571, 'quantity': 1646.0}..."


In [37]:
list_assets = df_portfolio.values[1][3]
list_assets

[{'asset': {'asset': 1571, 'quantity': 1646.0}},
 {'asset': {'asset': 1956, 'quantity': 73377.0}},
 {'asset': {'asset': 1958, 'quantity': 7420.0}},
 {'asset': {'asset': 2023, 'quantity': 748132.0}},
 {'asset': {'asset': 2024, 'quantity': 1999.0}},
 {'asset': {'asset': 2154, 'quantity': 62943.0}},
 {'asset': {'asset': 2062, 'quantity': 5212.0}},
 {'asset': {'asset': 2194, 'quantity': 285.0}},
 {'asset': {'asset': 2066, 'quantity': 11982.0}},
 {'asset': {'asset': 1778, 'quantity': 10376.0}},
 {'asset': {'asset': 2196, 'quantity': 330.0}},
 {'asset': {'asset': 2165, 'quantity': 102.0}},
 {'asset': {'asset': 1877, 'quantity': 94736.0}},
 {'asset': {'asset': 1912, 'quantity': 39049.0}},
 {'asset': {'asset': 1918, 'quantity': 69631.0}}]

In [38]:
d = [[list_assets[i]['asset']['asset'], list_assets[i]['asset']['quantity']] for i in range(len(list_assets))]

portfolio = pd.DataFrame(d, columns={'ASSET_DATABASE_ID', 'weights'})
portfolio

,ASSET_DATABASE_ID,weights
0,1571,1646.0
1,1956,73377.0
2,1958,7420.0
3,2023,748132.0
4,2024,1999.0
5,2154,62943.0
6,2062,5212.0
7,2194,285.0
8,2066,11982.0
9,1778,10376.0


In [39]:
our_assets = df_assets[df_assets['TYPE'] != 'PORTFOLIO']
our_assets = our_assets[['ASSET_DATABASE_ID', 'TYPE', 'LAST_CLOSE_VALUE_IN_CURR', 'MIN_BUY_AMOUNT', 'asset_fund_info_decimalisation']]

our_assets = our_assets.merge(portfolio, on='ASSET_DATABASE_ID')
our_assets

,ASSET_DATABASE_ID,TYPE,LAST_CLOSE_VALUE_IN_CURR,MIN_BUY_AMOUNT,asset_fund_info_decimalisation,weights
0,2154,FUND,17.866530473267,1276.8130745655,NaN,62943.0
1,2062,STOCK,101.302349336027,NaN,NaN,5212.0
2,2066,STOCK,82.116104868889,NaN,NaN,11982.0
3,1877,STOCK,8.785000000000,NaN,NaN,94736.0
4,1912,STOCK,19.650000000000,NaN,NaN,39049.0
5,1918,STOCK,4.630000000000,NaN,NaN,69631.0
6,2165,INDEX,10910.500000000000,NaN,NaN,102.0
7,1956,STOCK,9.770000000000,NaN,NaN,73377.0
8,1958,STOCK,145.950000000000,NaN,NaN,7420.0
9,2024,STOCK,100.000000000000,NaN,NaN,1999.0


In [40]:
def check_our_nav(assets):
  total_nav = np.dot(assets['weights'].values, assets['LAST_CLOSE_VALUE_IN_CURR'].values)
  print('TOTAL NAV = {}'.format(total_nav))

  length = assets.shape[0]
  print('nb assets = {}'.format(length))
  for i in range(length):
    print()
    nav = assets['LAST_CLOSE_VALUE_IN_CURR'].values[i] * assets['weights'].values[i]
    print('nav {} = {}'.format(assets['ASSET_DATABASE_ID'].values[i], nav))
    print('under than 10 % : {}'.format(nav <= total_nav * 0.1))
    print('more than 1 % : {}'.format(nav >= total_nav * 0.01))
  
check_our_nav(our_assets) 

TOTAL NAV = 11466159.811714578
nb assets = 15

nav 2154 = 1124573.0275788244
under than 10 % : True
more than 1 % : True

nav 2062 = 527987.8447393716
under than 10 % : True
more than 1 % : True

nav 2066 = 983915.1685390304
under than 10 % : True
more than 1 % : True

nav 1877 = 832255.76
under than 10 % : True
more than 1 % : True

nav 1912 = 767312.85
under than 10 % : True
more than 1 % : True

nav 1918 = 322391.52999999997
under than 10 % : True
more than 1 % : True

nav 2165 = 1112871.0
under than 10 % : True
more than 1 % : True

nav 1956 = 716893.2899999999
under than 10 % : True
more than 1 % : True

nav 1958 = 1082949.0
under than 10 % : True
more than 1 % : True

nav 2024 = 199900.0
under than 10 % : True
more than 1 % : True

nav 2023 = 314215.44
under than 10 % : True
more than 1 % : True

nav 1571 = 137622.06
under than 10 % : True
more than 1 % : True

nav 2194 = 1111509.703779033
under than 10 % : True
more than 1 % : True

nav 2196 = 1113230.3370783173
under than 10 % 

In [41]:
def check_our_stock(assets):
  total_nav = np.dot(assets['weights'].values, assets['LAST_CLOSE_VALUE_IN_CURR'].values)
  print('TOTAL NAV = {}'.format(total_nav))

  stock = assets[assets['TYPE'] == 'STOCK']
  length = stock.shape[0]
  print('nb stock assets = {}'.format(length))
  stock_nav = np.dot(stock['LAST_CLOSE_VALUE_IN_CURR'].values, stock['weights'].values)
  print('Stock NAV = {}'.format(stock_nav))
  print('stock more than 50 % : {}'.format(stock_nav >= total_nav * 0.5))
  
check_our_stock(our_assets) 

TOTAL NAV = 11466159.811714578
nb stock assets = 10
Stock NAV = 5885442.943278402
stock more than 50 % : True


In [42]:
def check_our_quantity(assets):
    for _, row in assets.iterrows():
      if ((row['MIN_BUY_AMOUNT'] is not np.NaN) and (row['weights'] < row['MIN_BUY_AMOUNT'])):
          return False
    return True

print('quantity respected : {}'.format(check_our_quantity(our_assets)))
our_assets[['MIN_BUY_AMOUNT', 'weights', 'asset_fund_info_decimalisation']]

quantity respected : True


,MIN_BUY_AMOUNT,weights,asset_fund_info_decimalisation
0,1276.8130745655,62943.0,NaN
1,NaN,5212.0,NaN
2,NaN,11982.0,NaN
3,NaN,94736.0,NaN
4,NaN,39049.0,NaN
5,NaN,69631.0,NaN
6,NaN,102.0,NaN
7,NaN,73377.0,NaN
8,NaN,7420.0,NaN
9,NaN,1999.0,NaN
